# Introducción.

La expresión QUBO significa Quadratic Unconstrained Binary Optimization. Es decir, se trataría de un modelo cuadrático sin restricciones (aunque ya veremos después que se pueden añadir) utilizado para la optimización de variables binarias (es decir variables que sólo pueden tomar dos valores, que serán el cero o el uno; o bien el -1 y el 1).

La tecnología que está detrás de todo esto, se basa en el comportamiento del electrón y el *spin*.El comportamiento magnético puede entenderse pensando en cada átomo como un  espín, que puede apuntar hacia arriba o hacia abajo.

Los espines pueden querer alinearse contra o con un campo magnético, de tal manera que los giros vecinos se afectan mutuamente, pero no ocurre lo mismo con los más alejados.

## El modelo ising.

Este modelo va tratar las variables como binarias pero que pueden tomar los valores -1 y +1. Entonces utilizando este modelo la función objetivo se va a definir de la siguiente manera:

$$
E=-\sum_{i}h_{i}S_{i}+\sum_{i,j}J_{ij}S_{i}S_{j}\qquad donde\:S_{i}\in\{-1,+1\}
$$

La variable se representa en este modelo por S ya que es la inicial de *Spin*.Los valores de h's y J's son constantes para ajustar por el modelo, una h para cada Spin y una J para cada interacción de los pares de Spines.

## El modelo QUBO.

En este caso el modelo se describe de la siguiente manera:

$$
Obj(c,a_{i},b_{ij},q_{i})=c+\sum_{i}a_{i}q_{i}+\sum_{i<j}b_{ij}q_{i}q_{j}
$$

En este caso las variables q's son variables de tipo qubit que valen 0 ó 1 en lugar de -1 ó 1

Para convertir una variable QUBO en Ising se utiliza la siguiente transformación:

$$
q_i=\frac{1+S_i}{2}
$$

Uno de los conceptos claves de esta tecnología es el D-Wave Quantum Processing Unit (QPU) que es el que se encaraga realmente de resolver los modelos Ising o QUBO . A este elemento se envían la QMI (Quantum Machine Instructions) para su procesamiento y entonces la QPU se encarga de localizar los estados de mínima energía es decir los valores de las variables donde se alcanza el mínimo de la función objetivo.

## Representación gráfica de un problema QUBO.

Los problemas de tipo QUBO se pueden representar mediante un esquema de tipo red. A continuación mostramos un ejemplo.

![](img/qubo.PNG/)

En la figura anterior podemos ver que en las aristas aparecen los términos cuadráticos y en los vértices los términos lineales. En concreto, en esta figura podemos ver la forma cuadrática que representa en la parte inferior de la misma. 

Los términos constantes y los nombres de las variables son normalmente omitidos en este tipo de representación.

## Representación en formato matricial.

Una forma más común de representación de los problemas de tipo QUBO es mediante una matriz, de tal manera que en la diagonal se ponen los elementos de los términos lineales y en la parte superior los coeficiente de los términos cuadráticos. Veamos el ejemplo anterior, cómo se dispondrá en forma matricial.

![](img/QUBOMatriz.PNG)

En el siguiente problema

$$ 
minimizar \, y=-5x_1-3x_2-8x_3-6x_4+4x_1x_2+8x_1x_3+2x_2x_3+10x_3x_4
$$

El término lineal será:

$$
-5x_1-3x_2-8x_3-6x_4
$$

Mientras que el término cuadrático es

$$
4x_1x_2+8x_1x_3+2x_2x_3+10x_3x_4
$$

Y la expresión matricial será

$$
\left[\begin{array}{cccc}
-5 & 4 & 8 & 0\\
0 & -3 & 2 & 0\\
0 & 0 & -8 & 10\\
0 & 0 & 0 & -6
\end{array}\right]
$$

Veamos cómo vamos a poder ir formalizando los problemas de tipo QUBO. Supongamos que disponemos de variables binaria (con valores 0 ó 1) y queremos favorecer valores iguales pero penalizar valores diferentes de las mismas. La expresión de la función objetivo sería:


$$
ax+by+cxy+d
$$

Entonces de acuerdo a estas características podemos construir la siguiente tabla.

$$
\left[\begin{array}{ccc}
X & Y & ax+by+cxy+d\\
0 & 0 & 0\\
0 & 1 & 1\\
1 & 0 & 1\\
1 & 1 & 0
\end{array}\right]
$$

Las dos columnas de las izquierda representan las posibles combinaciones de los valoress de las dos variables, y la tercera columna el resultado de acuerdo con las premisa inicial, es decir se tendrá menos energía cuando los dos valores coincida y más energía cuando los valores sean diferentes.

Entonces sustituyendo los valores de las variables  y haciéndolos igual a cero ( en la fórmula de la función objetivo se tiene).

$$
\left\{ \begin{array}{c}
0=d\\
1=b+d\\
1=a+d\\
0=a+b+c+d
\end{array}\right\} 
$$

De donde se obtiene

$$
\left\{ \begin{array}{c}
a=+1\\
b=+1\\
c=-2\\
d=0
\end{array}\right\} 
$$

Es decir

$$
x+y-2xy
$$

Con una representación gráfica igual a la siguiente

![](img/igualFavor.PNG)

Supongamos ahora un problema similar al anterior, pero por contra se penalizan los valores iguales y se favorecen los qubits con valores diferentes. Entonces en esta caso se tiene la siguiente matriz.

$$
\left[\begin{array}{ccc}
X & Y & ax+by+cxy+d\\
0 & 0 & 1\\
0 & 1 & 0\\
1 & 0 & 0\\
1 & 1 & 1
\end{array}\right]
$$

que daría lugar a las siguientes igualdades

$$
\left\{ \begin{array}{c}
1=d\\
0=b+d\\
0=a+d\\
1=a+b+c+d
\end{array}\right\} 
$$

De donde se obtiene

$$
\left\{ \begin{array}{c}
a=-1\\
b=-1\\
c=+2\\
d=1
\end{array}\right\} 
$$

En consecuencia la ecuación quedaría de la siguiente manera

$$
-x-y+2xy+1
$$

Y la expresión gráfica sería en este caso la siguiente:

![](img/IgualContra.PNG)

Un posible ejemplo para tres qubit se puede ver en la imagen siguiente

![](img/qubits3.PNG)

Veamos a modo de ejemplo, como podemos pasar a los ordenadores de Q-Wave el problema anterior, que en resumidas cuentas es equivalente a no tener en cuenta la constante y tomar como función objetivo $-x-y+2xy$. Entonces el programa en paython que escribiremos será el siguiente:

In [4]:
import dimod

exacsolver = dimod.ExactSolver()

Q={(0,0):-1,(1,1):-1,(0,1):2}

result = exacsolver.sample_qubo(Q)

#Sacamos los resultados
for sample, energy in result.data(['sample','energy']):
    print(sample,energy)

{0: 1, 1: 0} -1.0
{0: 0, 1: 1} -1.0
{0: 0, 1: 0} 0.0
{0: 1, 1: 1} 0.0


Como vemos las soluciones óptima son aquellas en que las dos variables valen igual, es decir las dos cero o las dos 1

## Planteamiento problemas en QUBO.

En este apartado vamos a dar algunas ideas sobre cómo poder plantear los problemas en QUBO. Imaginemos que tenemos cuatro personas con una cierta relación de amistad o enemistad como se muestra en la siguiente figura.

![](img/AmigosEnemigos.PNG)

Entonces de lo que se trata es obtener la una función objetivo de tal manera que nos represente perfectamente esa situación. Por lo tanto se deberán utilizar para ello variables binarias, que sólo tomen valores 1 ó 0.

La relación entre las dos persona X e Y y su expresión matemática la encontramos mediante la siguiente tabla

$$
\begin{array}{ccc}
X & Y & ax+by+cxy+d\\
0 & 0 & -1\\
0 & 1 & 0\\
1 & 0 & 0\\
1 & 1 & -1
\end{array}
$$

Que resolviendo como hemos hecho anteriormente nos lleva a la siguiente ecuación
$$
X+Y-2XY-1
$$

De forma análoga, para la relación Z con W obtenemos la siguiente ecuación

$$
Z+W-2ZW-1
$$

Y la relación de X con W se indicará de la siguiente manera

$$
X+W-2XW-1
$$

Veamos ahora cómo plasmar la relación de enemistad entre Z e Y.

$$
\begin{array}{ccc}
Z & Y & az+by+czy+h\\
0 & 0 & 0\\
0 & 1 & -1\\
1 & 0 & -1\\
1 & 1 & 0
\end{array}
$$

Que resolviéndolo nos lleva a la siguiente expresión

$$
-Y-Z+2XY
$$

Entonces sumando las cuatro expresiones obtenidas anteriormente obtendremos la función objetivo que vamos buscando:

$$
2X-2XY+2W-2ZW+2YZ-2XW-3
$$